# Interroger des documents (RAG)

In [ ]:
%pip install -qU wget openai

Commencez par télécharger le document qui va vous servir de base de connaissance.

In [27]:
# Download a file
import wget
import os

file_path = "my_document.pdf"
if os.path.exists(file_path):
    os.remove(file_path)
doc_url = "https://www.legifrance.gouv.fr/download/file/rxcTl0H4YnnzLkMLiP4x15qORfLSKk_h8QsSb2xnJ8Y=/JOE_TEXTE"

wget.download(doc_url, out=file_path)

'my_document.pdf'

Puis instancier la connexion à l'API Albert.

In [28]:
# OpenAI client configuration
import requests
from openai import OpenAI

base_url = "https://albert.api.etalab.gouv.fr/v1"
api_key = os.getenv("API_KEY")

client = OpenAI(base_url=base_url, api_key=api_key)

session = requests.session()
session.headers = {"Authorization": f"Bearer {api_key}"}  # skip headers if no api_key is setup in config.ini file

Enfin pour vous importer le document dans une collection de notre base vectorielle à l'aide du endpoint POST `/v1/files`.

Vous devez spécifier le modèle d'embeddings qui sera utilisé pour vectoriser votre document. Vous pouvez trouver la liste des modèles avec le endpoint `/v1/models`. Les modèles d'embeddings sont indiqués avec le type _feature-extraction_.

Le endpoint POST `/v1/files` doit retourner un status _success_. 

In [29]:
# Remove previous files

collection = "leo"
response = session.delete(f"{base_url}/files/{collection}")
response.status_code

204

In [30]:
# Upload a file
model = "intfloat/multilingual-e5-large"
params = {"collection": collection, "model": model} 

files = {'files': (os.path.basename(file_path), open(file_path, 'rb'), "application/pdf")}
response = session.post(f"{base_url}/files", params=params , files=files)

response.json()

{'object': 'list',
 'data': [{'object': 'upload',
   'id': 'b79ebff3-f5fe-4c18-9a40-483c4b1fb58d',
   'filename': 'my_document.pdf',
   'status': 'success'}]}

Vous pouvez observer les fichiers que vous avez importer dans une collection à l'aide du endpoint GET `/v1/files.`

In [31]:
# Retrieve the file ID for RAG
response = session.get(f"{base_url}/files/{collection}")
response.json()

{'object': 'list',
 'data': [{'object': 'file',
   'id': 'b79ebff3-f5fe-4c18-9a40-483c4b1fb58d',
   'bytes': 133606,
   'filename': 'my_document.pdf',
   'chunk_ids': ['0d820c13-9b3d-4f3c-a927-451b38a2b325',
    '1dfd8217-1c13-4015-ae05-99c0056aab71',
    '47b5fff8-ed95-4309-a900-6f5062608d22',
    '5c106595-f1dc-41ee-83e5-499f1b7cc241',
    '7d9efa5d-169b-496a-af92-be3a00cb526e',
    'a1caf15b-a2a5-40d2-a297-9c07542e672b',
    'a7b13d6d-351e-4cfa-b540-d6bf9ffaa398',
    'b0323349-4d8d-4286-a3c1-06e9691b1a4b',
    'b3e9ed3b-23ec-47f4-bc4b-7475c0d454f2',
    'c255a29f-ef4a-4968-91f9-4a86cde2204d',
    'c692d337-bc7f-49bc-90a1-c8d3e2c37f9e',
    'ccced44b-1ceb-47af-81ea-c1ea86da9349',
    'cddb2526-37b4-4306-b500-9776841c0496',
    'd96e59fd-2078-418c-9453-72528aaf3ff3',
    'deba4e35-a8df-4b01-ba74-3b80fb14b570',
    'ff274491-e187-4bf5-be70-fa1f2b25a3a6'],
   'created_at': 1722839924}]}

In [32]:
file_id = response.json()["data"][0]["id"]

In [33]:
# Display tools parameters
response = session.get(f"{base_url}/tools")
for tool in response.json()["data"]:
    if tool["id"] == "BaseRAG":
        print(tool["description"].strip())

Base RAG, basic retrival augmented generation.

    Args:
        embeddings_model (str): OpenAI embeddings model
        collection (Optional[List[str]], optional): List of collections to search in. Defaults to None (all collections).
        file_ids (Optional[List[str]], optional): List of file IDs in the selected collections (after upload files). Defaults to None (all files are selected).
        k (int, optional): Top K per collection (max: 6). Defaults to 4.
        prompt_template (Optional[str], optional): Prompt template. Defaults to DEFAULT_PROMPT_TEMPLATE.

    DEFAULT_PROMPT_TEMPLATE:
        "Réponds à la question suivante en te basant sur les documents ci-dessous : {prompt}

Documents :

{files}"


In [34]:
# Chat completions
data = {
    "model": "AgentPublic/llama3-instruct-8b",
    "messages": [{"role": "user", "content": "Qui est Ulrich Tan ?"}],
    "stream": False,
    "n": 1,
    "tools": [
        {
            "function": {
                "name": "BaseRAG",
                "parameters": {
                    "embeddings_model": model,
                    "collections": [collection],
                    "k": 2,
                },
            },
            "type": "function",
        }
    ],
}

response = client.chat.completions.create(**data)
print(response.choices[0].message.content)

Selon les documents, Ulrich Tan est le chef du pôle Datamin du département "Eatalab".


Vous pouvez récupérer les metadata des tools utilisées. Ces metadata vous donnera, entre autre, le prompt envoyé au modèle.

In [36]:
print(response.metadata[0]["BaseRAG"]["prompt"])

Réponds à la question suivante en te basant sur les documents ci-dessous : Qui est Ulrich Tan ?

Documents :

–  M. Gilles Corbi, agent contractuel, chef du pôle Production du département « ISO » ; –  M. Yann Brûlé, grade, chef du pôle « SI RIE » du département « ISO » ; –  M. Ulrich Tan, chef du pôle Datamin du département « Etalab » ; –  Mme Marie-Christie Ritz, cheffe du pôle « RH et attractivité » ; –  Mme Fadila Leturcq, cheffe du pôle « Campus du numérique ». 21  juin 2023 JOURNAL  OFFICIEL  DE  LA  RÉPUBLIQUE  FRANÇAISE Texte 5 sur 95

–  M.  Louis  di  Benedetto,  inspecteur  en  chef  de  la  santé  publique  vétérinaire,  chef  du  pôle  « Pilotage  de  la qualité et des partenariats » du département « ISO » ; –  M.  Philippe  Levillain,  agent  contractuel,  chef  du  pôle  « Réseau  interministériel  de  l’Etat  /  SOI »  du département « ISO » ;
